In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import PIL.Image as Image
from numpy import random,mat
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
import zipfile
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge

PREPROCESSING

In [8]:
#壓縮圖片
def compressImage(srcPath,dstPath):
    for filename in os.listdir(srcPath):           
        if not os.path.exists(dstPath):
                os.makedirs(dstPath)

        #完整的檔案或資料夾路徑
        srcFile=os.path.join(srcPath,filename)
        dstFile=os.path.join(dstPath,filename)

        # 如果是檔案就處理
        if os.path.isfile(srcFile):
            
             if os.path.isfile(srcFile):
                try:
                    sImg=Image.open(srcFile)
                    dImg=sImg.resize((100,100),Image.ANTIALIAS)  # 設定壓縮尺寸
                    dImg.save(dstFile)
                except Exception:
                    print(dstFile+"fail")

        # 如果是資料夾就遞迴
        if os.path.isdir(srcFile):
            compressImage(srcFile, dstFile)

In [9]:
compressImage("./data/Train/","./data/Train_compression/")     # 壓縮芒果圖片
path='./data/Train_compression/'
files=os.listdir(path)
files.sort()
img_list = []

#讀檔
for file in files:
    
    p=path+file
    img = cv2.imread(p)
    img_2D = np.reshape(img,(-1,3))
    img_1D = np.reshape(img_2D,(-1,1))
           
    img_1D = img_1D.flatten()
    img_1D = img_1D.tolist()

    img_list.append(img_1D)
img_arr = np.array(img_list)


compressImage("./data/Test/","./data/Test_compression/")     # 壓縮芒果圖片
path='./data/Test_compression/'
files=os.listdir(path)
files.sort()
test_list = []

#讀檔
for file in files:
    
    p=path+file
    img = cv2.imread(p)
    img_2D = np.reshape(img,(-1,3))
    img_1D = np.reshape(img_2D,(-1,1))
           
    img_1D = img_1D.flatten()
    img_1D = img_1D.tolist()

    test_list.append(img_1D)
test_arr = np.array(test_list)


print(test_arr)


[[112 101  87 ... 114 106  93]
 [ 43  62  67 ... 104 139 207]
 [168 175  84 ... 179 149 132]
 ...
 [123 127 128 ...  86  94 101]
 [ 80  92 104 ... 117 117 171]
 [ 95  99 100 ... 101 115 164]]


In [10]:
pca = PCA(n_components=100)
pca.fit(img_arr)
img_arr_pca = pca.transform(img_arr)

In [11]:
df = pd.read_csv('./data/Train.csv', index_col=0)
label_list = []
for i in range(len(df['label'])):
    if df['label'][i] == 'A':
        label_list.append(0)
    elif df['label'][i] == 'B':
        label_list.append(1)
    else:
        label_list.append(2)
print(len(label_list))

5600


SVC

In [10]:
clf = svm.SVC(kernel = 'poly', degree = 5, C = 0.9)
clf.fit(img_arr_pca, label_list)

SVC(C=0.9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

RANDOM FOREST

In [54]:
tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=1000, max_samples=0.8, random_state=1)
bag.fit(img_arr_pca, label_list)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.8, n_estimators=1000, n_jobs=1, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [ ]:
rf1 = RandomForestClassifier(n_estimators=1000,max_features=100)
rf1.fit(img_arr_pca, label_list)

In [27]:
rf2 = RandomForestClassifier(n_estimators=1000)
rf2.fit(img_arr_pca, label_list)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
rf3 = RandomForestClassifier(n_estimators=1000)
rf3.fit(img_arr_pca, label_list)

KNN

In [23]:
knn = KNeighborsClassifier(n_neighbors=10, weights='distance')
knn.fit(img_arr_pca, label_list)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='distance')

In [ ]:
LINEAR REGRESSION

In [8]:
#poly4 = PolynomialFeatures(degree=3)  # 4次多項式特征生成器
#img_arr_poly4 = poly4.fit_transform(img_arr_pca)
lasso_poly4 = Lasso()
lasso_poly4.fit(img_arr_poly4, label_list)

NameError: name 'img_arr_poly4' is not defined

VALIDATION

In [ ]:
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle = True, random_state=7)
results = sklearn.model_selection.cross_val_score(rf1, img_arr_pca, label_list, cv=kfold)
print(results.mean())

In [16]:
test_arr_pca = pca.transform(test_arr)

PREDICT

In [21]:
predict1 = clf.predict(test_arr_pca)
print(predict1)

[1 0 2 2 0 2 1 2 0 2 2 0 1 0 0 2 1 1 1 1 0 1 2 1 1 2 1 1 1 0 0 0 1 2 2 1 0
 1 1 1 0 0 2 1 1 1 2 1 1 1 1 2 1 1 2 2 0 1 2 0 0 1 2 1 1 1 1 1 1 0 1 1 1 0
 1 1 2 2 1 2 2 1 1 2 1 1 1 0 1 2 0 1 2 0 0 1 1 0 1 1 0 2 0 1 2 1 0 0 1 0 0
 1 0 2 1 2 0 0 1 1 1 2 0 2 0 0 1 2 1 0 2 2 2 0 2 2 1 1 1 1 1 2 1 0 2 0 1 0
 1 2 1 1 1 2 2 1 1 1 1 1 1 1 0 1 2 1 2 0 0 1 2 1 1 1 1 1 1 1 2 0 2 0 2 0 2
 2 0 0 0 1 2 0 0 0 0 1 1 0 1 2 2 2 0 2 0 1 2 1 1 1 2 2 2 1 2 1 1 1 0 2 1 2
 0 0 0 0 2 2 0 1 0 0 1 1 0 2 1 1 0 1 2 0 0 1 2 0 1 1 1 1 2 2 1 0 1 1 2 1 1
 1 1 0 2 2 0 1 0 1 2 2 1 2 0 1 1 1 2 1 1 1 1 0 2 0 0 1 2 1 1 2 1 2 1 1 0 2
 2 1 0 1 0 0 2 0 2 2 0 2 1 1 1 1 0 2 2 0 2 0 1 1 2 1 2 1 1 1 0 1 1 0 1 1 0
 0 0 0 0 1 0 2 0 0 2 0 0 1 0 1 1 1 2 1 1 1 1 2 0 1 1 0 1 0 0 1 1 1 1 2 2 2
 2 0 2 1 1 2 2 1 2 0 2 2 1 0 1 2 2 0 1 1 2 1 2 0 0 1 1 2 1 0 0 2 1 1 1 2 1
 2 0 1 0 1 1 0 1 2 0 0 1 0 1 2 1 1 0 2 0 1 0 0 2 0 2 0 1 1 2 0 1 2 1 1 2 0
 1 2 1 0 1 1 2 1 2 1 0 0 0 2 1 0 0 0 0 1 2 2 2 0 2 0 1 0 1 1 1 1 1 2 2 1 2
 1 0 1 0 2 1 2 1 0 1 0 0 

In [47]:
predict2 = bag.predict(test_arr_pca)
print(predict2)

[1 0 2 2 0 2 1 1 0 2 2 0 1 2 0 2 1 1 2 1 0 1 2 0 2 2 2 1 1 0 0 1 1 2 0 1 0
 1 0 1 0 0 2 1 0 2 1 2 0 0 1 1 1 2 2 1 0 1 2 0 0 1 1 2 2 1 1 2 1 1 1 1 1 0
 1 0 2 2 1 2 2 0 1 2 1 1 2 0 1 2 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 0
 1 0 1 0 2 0 0 1 0 0 2 0 2 0 0 1 2 1 0 1 0 1 2 2 1 1 1 2 2 1 2 2 0 2 1 0 0
 2 2 1 1 1 2 2 1 2 1 0 1 1 1 0 1 1 2 2 0 0 1 1 1 1 1 1 1 1 0 2 0 2 0 1 0 2
 2 0 0 0 2 2 0 0 0 0 2 1 0 0 2 0 2 0 2 0 2 2 1 1 1 2 2 2 1 2 1 1 1 0 1 1 2
 0 0 0 0 2 2 0 1 2 0 0 1 1 1 1 0 0 1 2 1 0 1 2 0 2 1 1 1 2 2 1 0 1 2 2 1 1
 0 1 0 2 2 1 2 0 2 1 2 1 2 0 1 0 1 2 2 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 0 0
 2 1 1 0 0 0 2 0 2 1 0 2 0 1 1 1 0 2 2 0 1 1 0 1 0 1 1 1 2 2 2 1 1 0 2 1 0
 0 0 1 0 1 0 2 0 0 2 0 1 1 0 1 0 1 1 2 1 1 1 1 0 0 2 0 0 0 0 1 0 1 1 0 2 2
 2 0 1 1 1 2 2 2 2 1 2 0 1 0 1 2 1 0 1 0 1 1 0 0 0 1 0 2 1 0 0 2 2 1 1 2 1
 1 1 1 2 2 1 0 1 2 0 0 1 0 1 1 2 2 1 2 0 1 0 0 2 1 2 0 1 0 0 0 1 2 1 1 2 1
 0 0 1 0 0 1 2 2 2 1 0 2 0 1 1 2 0 0 0 0 0 2 2 0 2 0 1 0 1 0 1 0 1 2 2 1 2
 1 0 1 0 2 0 2 2 0 1 1 0 

In [22]:
predict3 = rf1.predict(test_arr_pca)
print(predict3)

[1 0 2 2 0 0 1 1 0 2 2 0 1 2 0 1 1 1 1 1 0 1 2 0 2 2 1 1 0 0 0 1 1 2 0 1 0
 1 0 1 2 0 1 1 0 2 1 2 0 0 1 1 1 2 2 2 0 1 2 0 0 1 1 1 2 1 1 1 1 0 1 2 2 0
 1 0 2 2 1 2 2 0 1 2 2 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 0
 1 0 1 0 2 0 0 1 0 0 2 0 2 0 0 2 2 1 0 1 0 1 0 2 1 1 1 2 2 1 1 2 0 2 1 0 0
 2 2 1 1 1 2 2 1 2 1 0 1 1 1 0 1 2 2 2 0 0 1 1 1 1 1 1 1 1 0 2 0 2 0 2 0 2
 2 0 0 0 2 2 0 0 0 0 2 1 0 0 2 0 2 0 2 0 2 2 1 1 1 1 1 2 1 2 1 1 1 0 2 1 2
 0 0 0 0 2 2 0 1 2 0 0 1 1 1 1 0 0 1 2 1 0 1 2 0 2 1 1 1 2 2 1 0 1 1 2 1 1
 1 1 0 2 2 1 2 1 2 1 2 1 2 0 1 0 1 2 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 0 0
 2 1 1 0 0 0 2 0 2 1 0 2 0 0 1 1 0 2 2 0 1 0 0 1 0 1 1 1 1 2 1 1 1 0 2 1 0
 0 1 1 0 1 0 2 0 0 1 0 1 1 1 1 1 1 1 2 1 1 1 1 1 0 2 0 0 0 0 1 0 1 1 2 2 2
 2 0 1 1 1 2 2 2 2 1 2 0 1 0 1 2 1 0 1 0 2 1 0 0 0 1 0 2 1 0 0 2 2 1 1 2 1
 1 0 1 2 2 1 0 1 1 0 0 1 0 1 1 2 2 1 2 0 0 0 0 2 0 2 0 1 0 2 0 1 2 1 1 2 0
 0 0 1 0 0 1 1 2 2 1 0 0 0 2 1 1 0 0 0 0 0 1 2 0 2 0 1 0 1 0 2 0 1 2 2 1 2
 1 0 1 0 2 1 2 2 0 1 1 0 

In [23]:
predict4 = rf2.predict(test_arr_pca)
print(predict4)

[1 0 2 2 0 0 1 1 0 2 2 0 1 2 0 2 1 1 1 1 0 1 2 0 1 2 1 1 1 0 0 1 1 2 0 1 0
 1 0 1 2 0 1 1 0 2 1 1 0 0 1 1 1 2 2 2 0 1 2 0 0 1 1 1 1 1 1 1 1 0 1 2 2 0
 1 0 2 2 1 2 2 0 1 2 2 1 2 0 1 2 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 0
 1 0 1 0 2 0 0 1 0 1 2 0 2 0 0 1 2 1 0 1 0 1 0 2 1 1 1 2 2 1 2 2 0 2 1 0 0
 2 2 1 1 1 2 2 1 2 1 0 1 1 1 0 1 2 2 2 0 0 1 1 1 1 1 1 1 1 0 2 0 2 0 2 0 2
 2 0 0 0 2 2 0 0 0 0 2 1 0 0 2 0 2 0 2 0 2 2 1 1 1 1 1 2 1 2 1 1 1 0 2 0 2
 0 0 0 0 2 2 0 1 2 0 0 1 1 1 1 0 0 1 2 1 0 1 1 0 2 1 1 1 2 2 1 0 1 1 2 1 1
 0 1 0 2 2 1 2 1 1 1 2 1 2 0 1 0 1 2 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 2 0 0
 2 1 1 0 0 0 2 0 2 1 0 2 0 1 1 1 0 2 2 0 1 0 0 1 1 1 1 1 1 2 1 1 1 0 2 1 0
 0 0 1 0 1 0 2 0 0 1 0 1 1 0 1 0 1 1 2 1 1 1 1 0 0 2 0 0 0 0 1 0 1 1 2 2 2
 2 0 1 1 1 2 2 2 2 1 2 0 1 0 1 2 1 0 1 0 0 1 0 0 0 1 0 2 1 0 0 2 2 1 1 2 1
 1 0 1 2 2 1 0 1 1 0 0 1 0 1 2 1 1 1 2 0 0 0 0 2 0 2 0 1 0 2 0 1 2 1 1 2 1
 0 1 1 0 0 1 2 1 2 1 0 0 0 1 1 1 0 0 0 0 0 1 2 0 2 0 1 0 1 0 1 0 1 2 2 1 2
 1 0 1 0 2 0 2 2 0 1 1 0 

In [56]:
predict5 = rf3.predict(test_arr_pca)
print(predict5)

[1 0 2 2 0 0 1 1 0 2 2 0 1 2 0 2 1 1 1 1 0 1 2 0 1 2 1 1 1 0 0 1 1 2 0 1 0
 1 0 1 1 0 2 1 0 2 1 2 1 0 1 1 1 1 2 2 0 1 2 0 0 1 1 1 2 0 1 1 1 0 1 2 1 0
 1 0 2 2 1 2 2 0 1 2 2 1 2 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0
 1 0 1 2 2 0 0 1 0 1 2 0 2 0 0 1 2 1 0 1 0 1 0 2 1 1 1 1 2 1 2 2 0 2 1 0 0
 2 2 1 1 1 2 2 1 2 1 2 1 1 1 0 1 2 2 2 0 0 1 1 1 1 1 1 1 1 0 2 0 2 0 2 0 2
 2 0 0 0 2 2 0 0 0 0 2 1 0 0 2 0 2 0 2 0 2 2 1 1 1 1 1 2 1 2 1 1 1 0 2 1 2
 0 0 0 0 2 2 0 1 2 0 0 1 1 1 1 0 0 1 2 1 0 1 2 0 2 1 1 1 2 2 1 0 1 2 2 2 1
 0 1 0 2 2 1 2 0 2 1 2 1 2 0 1 0 1 2 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 0 0
 2 1 1 0 0 0 2 0 2 1 0 2 0 1 1 1 0 2 2 0 1 0 0 1 0 1 1 1 1 2 1 1 1 0 1 1 0
 0 1 1 0 1 0 2 0 0 1 0 1 1 0 1 1 1 1 2 1 1 1 1 1 0 2 0 0 0 0 1 0 1 1 2 2 2
 2 0 1 1 1 2 2 2 2 1 2 0 1 0 1 2 2 0 1 0 2 1 0 0 0 1 0 2 1 0 0 2 2 1 1 2 1
 1 0 1 2 2 1 0 1 1 0 0 1 0 1 1 2 1 1 2 0 0 0 0 2 0 2 0 1 0 2 0 1 2 1 1 2 1
 0 1 1 0 0 1 1 1 2 1 0 0 0 2 1 0 0 0 0 0 0 2 2 0 2 0 1 0 1 0 1 0 1 2 2 1 2
 1 0 1 0 2 0 2 2 0 1 1 0 

In [17]:
predict6 = knn.predict(test_arr_pca)
print(predict6)

[1 0 1 2 0 2 1 1 0 2 2 1 1 2 0 2 1 1 2 1 0 1 2 2 1 2 0 1 0 0 0 2 1 2 2 1 0
 1 1 0 2 0 2 1 1 2 1 1 1 0 1 2 1 2 2 1 0 1 2 0 0 1 1 1 2 0 1 1 1 0 1 2 1 0
 0 0 2 2 1 2 0 0 2 2 1 1 1 2 1 2 0 1 2 0 0 1 2 1 0 1 1 2 0 1 1 1 0 0 1 0 0
 1 0 2 2 2 0 0 1 1 1 2 0 2 0 1 2 2 0 0 0 2 1 0 2 1 1 1 2 2 1 2 2 0 2 1 2 0
 2 2 2 1 1 2 2 1 2 0 2 1 1 1 0 2 2 2 2 0 0 1 2 2 2 2 1 1 1 1 2 0 2 0 2 0 2
 2 0 0 0 2 2 0 0 0 0 2 1 0 0 2 0 1 0 2 0 2 2 1 1 2 2 2 2 2 2 2 1 0 0 2 0 2
 0 0 0 2 2 2 0 1 0 0 0 1 1 1 1 1 0 1 2 1 0 2 2 0 0 1 1 0 2 2 1 0 0 2 0 0 1
 0 1 0 2 2 0 2 0 2 2 2 1 2 0 1 0 1 2 1 1 1 1 0 2 0 0 1 2 1 1 2 1 1 1 2 0 0
 2 1 0 0 0 0 2 0 2 0 0 2 1 1 1 1 0 2 2 0 2 0 1 2 2 1 1 1 1 1 1 1 1 0 2 2 0
 0 1 0 0 1 0 2 0 0 2 0 0 1 0 1 0 1 1 2 0 1 1 1 1 1 2 0 0 0 0 1 0 0 1 2 2 2
 2 2 1 2 0 2 1 2 2 1 1 2 1 0 1 2 2 0 1 0 1 1 0 0 0 1 0 2 0 0 0 2 2 1 0 2 1
 2 0 1 1 2 1 1 1 2 0 0 1 0 1 2 2 1 0 2 0 0 0 1 2 0 2 0 1 0 0 0 1 2 2 2 2 0
 0 2 1 0 0 1 2 1 2 1 1 0 0 2 1 0 1 0 0 0 1 2 2 0 2 0 1 0 1 2 1 0 1 2 2 1 2
 1 0 1 0 2 0 2 1 0 0 1 0 

In [ ]:
predict7 = lasso_poly4.predict(test_arr_pca)
print(predict7)

In [2]:
df = pd.read_csv('./data/0610832_result5.csv', index_col=0)
label5_list = []
for i in range(len(df['label'])):
    if df['label'][i] == 0:
        label5_list.append(0)
    elif df['label'][i] == 1:
        label5_list.append(1)
    else:
        label5_list.append(2)
print(label5_list)

[1, 0, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 2, 1, 1, 0, 0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 0, 1, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 1, 2, 0, 1, 2, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 1, 1, 2, 1, 2, 2, 0, 2, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 0, 1, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 1, 0, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 1, 1, 1, 0, 2, 2, 0, 1, 0, 0, 2, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 

In [3]:
df = pd.read_csv('./data/0610832_result11.csv', index_col=0)
label11_list = []
for i in range(len(df['label'])):
    if df['label'][i] == 0:
        label11_list.append(0)
    elif df['label'][i] == 1:
        label11_list.append(1)
    else:
        label11_list.append(2)
print(label11_list)

[1, 0, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0, 1, 2, 0, 2, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 1, 1, 1, 0, 0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 1, 1, 0, 2, 1, 1, 0, 0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 0, 1, 0, 2, 2, 1, 2, 2, 0, 1, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 0, 1, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 1, 2, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 0, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 1, 1, 1, 0, 2, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 0, 

In [4]:
df = pd.read_csv('./data/0610832_result13.csv', index_col=0)
label13_list = []
for i in range(len(df['label'])):
    if df['label'][i] == 0:
        label13_list.append(0)
    elif df['label'][i] == 1:
        label13_list.append(1)
    else:
        label13_list.append(2)
print(label13_list)

[1, 0, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0, 1, 2, 0, 2, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 1, 1, 1, 0, 0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 0, 1, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 1, 1, 2, 1, 2, 2, 0, 2, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 0, 1, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 1, 0, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 1, 0, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 1, 1, 1, 0, 2, 2, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 

多數決

In [9]:
predict_final = []
for i in range(len(predict1)):
    if predict1[i] == predict3[i]:
        predict_final.append(predict1[i])
    elif predict1[i] == predict4[i]:
        predict_final.append(predict1[i])
    elif predict3[i] == predict4[i]:
        predict_final.append(predict4[i])
    else:
        predict_final.append(predict4[i])


NameError: name 'predict1' is not defined

In [5]:
predict_final = []
for i in range(len(label5_list)):
    if label5_list[i] == label11_list[i]:
        predict_final.append(label5_list[i])
    elif label5_list[i] == label13_list[i]:
        predict_final.append(label5_list[i])
    elif label11_list[i] == label13_list[i]:
        predict_final.append(label11_list[i])
    else:
        predict_final.append(label5_list[i])

寫檔

In [6]:
df = pd.read_csv('./data/Train.csv', index_col=0)
with open ('0610832_result.csv','w') as f:
    f.write('ID,label\n')
    for i in range(len(predict_final)):
        f.write('%04d.jpg,%d\n'%(i, predict_final[i]))